In [ ]:
# Import standard libraries
import os
import io
os.environ["OMP_NUM_THREADS"] = '1'
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as map
%matplotlib inline
import time
import panel as pn
from panel.interact import interact, fixed

# ML import
from sklearn import datasets        # datasets
from sklearn.cluster import KMeans  # K-Means algorithm
from sklearn.cluster import AgglomerativeClustering  # Hierarchical clustering
from scipy.cluster.hierarchy import dendrogram, linkage # dendogram visualization
from sklearn.preprocessing import LabelEncoder #Label encoding
from sklearn.preprocessing import OneHotEncoder # 1-hot encoding

#import from mlxtend for Association rule
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#Import from surprise library
from surprise import Dataset
from surprise import get_dataset_dir
from surprise import KNNBasic, KNNWithMeans, SVD
from surprise.model_selection import train_test_split 
from surprise.model_selection import cross_validate, GridSearchCV
from collections import defaultdict

# Assignment 2

Welcome to the second assignment! 

You will have to implement clustering, association rules, and recommender systems algorithms, applying these methods to: 
- explore the similarities within groups of people watching movies (clustering analysis)
- discover the relations between movies genre (association rules)
- recommend movies to users (recommender system)

We will use the MovieLens dataset, which contains movie ratings collected from the MovieLens website by the [GroupLens](https://grouplens.org/) research lab.

Source: F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. *ACM Transactions on Interactive Intelligent Systems (TiiS)* 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

Once you are done you have to submit your notebook here: 
[https://moodle.epfl.ch/mod/assign/view.php?id=1247726](https://moodle.epfl.ch/mod/assign/view.php?id=1247726)

If there is need for further clarifications on the questions, after the assignment is released, we will update this file, so make sure you check the git repository for updates.

Good luck!

## Clustering analysis: similarities between people (10 points)

In this section, you will try to form clusters of individuals based on their preferences regarding movie genres. You will use a transformed version of the MovieLens dataset containing, for a selection of users:
- their average rating of all science fiction movies they rated,
- their average rating of all comedy movies they rated.

Better understanding the differences in people's tastes can help improve the design of recommender systems, for instance for the creation of the user neighborhood. Ok, let's start!

- Load the data in a dataframe. The url link is provided below. Display the first 10 observations.

In [ ]:
#project_dir = r"C:\Users\Charlotte Ahrens\Documents\Personal\GitHub\MGT-502-Data-Science-and-Machine-Learning\data"

In [ ]:
#load and display data frame
url_clustering = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/ratings_clustering.csv'
#url_clustering = os.path.join(project_dir,"ratings_clustering.csv")
df_rating = pd.read_csv(url_clustering)
display(df_rating.head(10))
display(df_rating.describe())   

- Plot a Dendrogram using "ward" as linkage method and "euclidean" as metric. 
- Based on the Dendrogram, how many clusters do you think is optimal? Briefly justify your answer.

In [ ]:
# plot dendrogram
dendo_rating = linkage(df_rating, method="ward", metric="euclidean")

plt.figure(figsize=(10,8))
dendrogram(dendo_rating, show_leaf_counts=False)
plt.title("Dendrogram - individual's movie rating", fontsize= 16)
plt.xlabel("Individuals", fontsize= 13)
plt.tick_params(axis = "x", labelbottom=False)
plt.ylabel("Distance", fontsize= 13)
plt.hlines(y= 4.5, xmin=0, xmax=2000, colors= "k", linestyle= "--")
plt.text(y= 4.6, x= 1000, s= "Horizontal line crossing 4 vertical lines")
plt.show()


### Explanation (in green)
<font color= darkgreen> 


Based on the above Dendrogram, I would expect 4 clusters to be the optimal number to cluster the individuals based on their movie genre taste. I determined 4 clusters by looking where I can find the longest vertical line that is not cut by a horizontal line. This distance can be found from ~ y=4 to ~ y=5.8 (Total distance: ~1.8) very close to this are 2 clusters since here we are having a distance from ~ y=5.8 to ~ y=7.2 (Total distance= ~1.4). 

- Implement the Elbow method to determine the optimum number of cluster for K-Means algorithm (use `random_state=17` as parameter of K-Means). 
- Based on the Elbow method, how many clusters do you think is optimal? Briefly justify your answer.

In [ ]:
#elbow method for cluster determination 
inertias = []

poss_clusters = range(1,10)
for i in poss_clusters:
    km = KMeans(n_clusters = i, random_state = 17, n_init="auto")
    km.fit(df_rating)
    inertias.append(km.inertia_)

plt.plot(poss_clusters, inertias, "-x")
plt.vlines(x= 4, ymin=0, ymax= 120, linestyle= "--", colors="k")
plt.text(x= 4.2, y= 100, s= "potential optimum at 5 clusters")
plt.xlabel("Number of cluster")
plt.ylabel("Inertia")
plt.title("Elbow method for inertia")
plt.show()


### Explanation
<font color= darkgreen> 


The elbow method confirms the previous finding of an optimal of 4 clusters. The Inertia measures how spread out the individuals ratings are within one cluster. The inertia decreases with an increasing # of clusters since the variance within the clusters decreases. Until 4 clusters, the inertia decreases, characterized by a steep slope, while from 4 clusters onwards the slope flattens. The "elbow" shows 4 to be the optimal cluster #.

- Implement (train) a K-Means algorithm with the number of clusters of your choice. Use `random_state=17` as parameter.

In [ ]:
# K-means algorithm with random state n = 17
kmeans4 = KMeans(n_clusters = 4, random_state = 17, n_init= "auto")
kmeans4.fit(df_rating)
display(kmeans4.cluster_centers_)     
display(kmeans4.labels_)                                            

- Implement (train) a hierarchical algorithm with the same number of clusters as for the K-Means model. Use "ward" as linkage method and "euclidean" as metric/affinity 

In [ ]:
# Hierarchical clustering with linkage = ward and metric = euclidean
hierarchical4 = AgglomerativeClustering(n_clusters = 4, linkage = "ward", metric = "euclidean")
hierarchical4.fit(df_rating)
display(hierarchical4.labels_)    


- Create a figure consisting of two subplots:
    - a scatterplot of 'avg_scifi_rating' and 'avg_comedy_rating' colored by the clusters predicted with your KMeans model. Add the cluster centers to your plot. Label your clusters with the name of your choice (e.g., "Comedy aficionado").
    - a scatterplot of 'avg_scifi_rating' and 'avg_comedy_rating' colored by the clusters predicted with your hierarchical algorithm model. Label your clusters with the name of your choice.
- How do your models compare?

In [ ]:
df_rating

In [ ]:
# renumber hierarchical data labels so the labels in both algorithm refer to the same labeled clusters 
dictionary = cluster_relabelling = {2:2, 3:0, 0:3, 1:1}
cluster_h = [cluster_relabelling[label] for label in hierarchical4.labels_]

# add cluster labels to the original data frame df_rating
df_rating["cluster_km"] = kmeans4.labels_
df_rating["cluster_h"] = cluster_h

In [ ]:
# compare kMeans and hierarchical model by creating two subplots
fig, ax =  plt.subplots(1,2, figsize= (10,8)) # creating 2 subplots next to another (1 row, 2 columns)
fig.suptitle("Distribution of individual's movie rating")
lg_labels =["Indifferent","Comedy lovers","Imagination geeks","Fun over Imagination"]

#plotting the 1st plot with k_means clusters
km_scatter = ax[0].scatter(x= df_rating["avg_scifi_rating"], # x-axis
            y= df_rating["avg_comedy_rating"],# y-axis
            c = df_rating["cluster_km"],# split data into clusters
            cmap = plt.colormaps["Set3"], #color each cluster differently
            marker = "8") # varies the market style between the clusters
ax[0].set_title("K-Means with 4 clusters") # title
ax[0].set_xlabel("Science fiction") # x-axis
ax[0].set_ylabel("Comedy") # y-axis
ax[0].legend(handles=km_scatter.legend_elements()[0], labels=lg_labels)

#adding cluster centers
ax[0].scatter(kmeans4.cluster_centers_[:,0],
              kmeans4.cluster_centers_[:,1],
              c= "black",
              marker = "X")


#plotting the 2nd plot with hierarchical clusters
h_scatter = ax[1].scatter(x= df_rating["avg_scifi_rating"], # x-axis
            y= df_rating["avg_comedy_rating"],# y-axis
            c = df_rating["cluster_h"], # split data into clusters
            cmap = plt.colormaps["Set3"], #color each cluster differently
            marker = "8") #varies the market style between the clusters
ax[1].set_title("Hierarchical clustering with 4 clusters") #title
ax[1].set_xlabel("Science fiction") # x-axis
ax[1].set_ylabel("Comedy") # y-axis
ax[1].legend(labels= lg_labels)
ax[1].legend(handles=h_scatter.legend_elements()[0], labels=lg_labels)


plt.show()

### Explanation:
<font color= darkgreen> 

Looking at the two graphs above, we see that we are comparing two scatterplots with the same amount of clusters, while the graphs differentiate themselves with the underlying algorithm that computes the individuals into clusters. Starting with the left plot, we observe the cluster "Indifferent" and "Fun over Imagination" to spark out as they seem to be the clusters with the most observations of all four. On the contrary the cluster "Imagination geeks" as well as "Comedy lovers" stand out, due to their outliers at the bottom right and top left, and their few observations. In general the main center of observations is captured by "Indifferent" at x = 3 andy = 3 and "Fun over Imagination" at x=3 and y= 3.5.

Comparing this with the hierarchical clustering we find a more equal distribution of the center between "Indifferent", "Fun over Imagination", and "Imagination geeks". Especially "Indifferent" lost many of its observations to the grey and the yellow colored clusters and is therefore comparably small. Also the blue colored cluster "Comedy lovers" lost some of its already few observations, mainly to the yellow cluster. 

The differences in cluster size and distribution is seen due to the different algorithms that we apply. The k_means model, is an iterative model that starts with the given number of cluster centers and assigns each data point to the nearest center. It then updates the centers to the mean of the data points assigned to them, repeating this process until teh centers are not moving anymore. While the hierarchical /agglomerative clustering doesn't require a given center number (we determined this one through the elbow method) and it is considering each data point as a separate cluster and then merges clusters based on their similarity. To mention is that in both cases the algorithm tries to reduce the euclidean distance between the data points, while additionally the hierarchical cluster centers are calculated by the ward linkage. 

## Association Rules: association between movie genres (10 points)

You will now pursue your analysis, but this time trying to dig out information about movies. More precisely, you will search for matches between film genres using association rules. We try to understand, for instance, how likely it is that a film is both drama and action. This information can be interesting for film producers who may either want to produce something similar to the established norm: if most drama films are also action, perhaps the new action-drama film would be equally appreciated, or quite to the contrary try a new combination of genres which is more rare to find.

- Load the data in a dataframe. The url link is provided below. 
- Display the first 10 observations. 
- Print the unique values of genres from the first column. 
- How many unique genres does the first column contain? 
- How many movies does the dataframe contains?

In [ ]:
#load and display movie genre dataframe
url_association_rules = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/movies_assoc_rules.csv'
#url_association_rules = os.path.join(project_dir,"movies_assoc_rules.csv")
df_movies = pd.read_csv(url_association_rules)
display(df_movies.head(10))
display(df_movies.describe())

#print the unique genres
print(df_movies["0"].unique(),'\n')
print("The first column of the genre dataframe contains %d different genres." % len(df_movies["0"].unique()))

#double check that all genres were displayed in the first column.
# convert the movie data frame to a list 
movie_list = df_movies.values.tolist()

# delete all the nan'sets and just key genres in the list
movie_list_cleaned = [[x for x in row if str(x) != "nan" ] for row in movie_list] # if my dataset would be numerical I could use: not math.isnan(x)
total_movies_list = [x for y in movie_list_cleaned for x in y]

#display the individual lists of above in one big list to find unique values:
total_movies = set(total_movies_list)
print("The total number of different genres in the dataframe is %d." % len(total_movies))

#Total number of movies
non_nan = df_movies["0"].count().sum()
print("The dataframe contains in total %d movies." %non_nan)


- Preprocessing: as seen during the lab, convert the dataset using a `Transaction Encoder` from the `mlextend` module so that the dataset is reorganised in columns of unique genres. Rows should contain only True or False boolean values according to whether a film was considered as belonging to a genre column or not. Check that you have the correct dimensions.

In [ ]:
#create the encoder
te = TransactionEncoder()

#fit and transform the data
movie_list_encoded= te.fit(movie_list_cleaned).transform(movie_list_cleaned)

#create dataframe with type of genre as header
df_movie_encoded = pd.DataFrame(movie_list_encoded, columns=te.columns_)
display(df_movie_encoded.head(10))
print("The dataframe contains %d observations and %d different genres." %(len(df_movie_encoded[0:]),df_movie_encoded[:1].sum().count()))

- Frequent itemsets: using the Apriori algorithm to find the frequent itemsets with minimum support of 0.01. There is no condition on the maximum length of an itemset. 
- How many itemsets did the apriori algorithm return above (for min_support=0.01)? 
- What are the 10 itemsets with the largest support (you can directly display a dataframe with the 10 itemsets and their support)?

In [ ]:
#determine frequent itemsets with min support 0.01 and no max_len
freq_items = apriori(df_movie_encoded, min_support= 0.01, use_colnames= True)
display(freq_items.sort_values(by="support",ascending=False).head(10))
display("With a minimum support of 0.01, the Apriori algorithm returned %d itemsets." %len(freq_items["support"]))

The itemseets with the largest support are Drama with a support on fearly 0.5. It is followed by Comedy with a support of 0.31. After, thriller and romance are following with a very similar support at ~0.15.

- Mining for association rules: using the frequent items identified above, find association rules with a minimum confidence of 0.45 and order them by decreasing value of lift.
- Discuss the following statements (true or false with 1-2 lines justification)
    - Animation films are associated with Children.  
    - If a film has the genre Musical, then it is also a Comedy.
    - If War then Drama is the asociation rule with the highest confidence.  

In [ ]:
# mine for association rules with confidence >= 0.45

rules_genre = association_rules(freq_items, metric = "confidence", min_threshold = 0.45)
rules_genre = rules_genre.sort_values(by = "lift", ascending=False)
display(rules_genre)
Musical_relations = rules_genre[rules_genre["antecedents"].astype(str).str.contains("Musical")]
Musical_relations

### Discussion

<font color= darkgreen> 

1. Animation films are associated with Children. True. 

Due to its highest lift in this dataset with ~10.9. Lift measures the strength of the association between the antecedent and the consequent and therefore the above rule is valid, that when we have Animation movies, the likelihood of it being also a children genre is very high. 

2. If a film has the genre Musical, then it is also a Comedy. False.

 There is an association, meaning a relationship between these two genres, but not a equalization or implication. In addition the rule is quite weak with a lift of 1.32 and therefore even a relationship is not strong between Music and Comedy.

3. If War then Drama is the association rule with the highest confidence. True.

If war is the antecedent, then the consequent "Drama" has a confidence of ~0.749. In this list this is the highest confidence, followed by "if Romance, then Drama" with a confidence of 0.623.


## Recommender systems: item-based recommender system (10 points)

In the walkthrough, we have implemented a user-to-user collaborative filtering algorithm (from scratch and using using Surprise library), i.e., our recommendations were based on the ratings of users with similar tastes. In this assignment, you will implement an **item-to-item** collaborative filtering algorithm, i.e., the recommendations will be based on the set of movies that users like. Do not worry, you won't have to implement the algorithm from scratch and instead can rely on the [Surprise library](http://surpriselib.com/). 

- As in the walkthrough, load the *built-in* `ml-100k` from the Surprise library.

In [ ]:
#Load the builtin dataset
data_recomm = Dataset.load_builtin("ml-100k")

- Use GridSearchCV to find the best number of neighbors (k) for a KNNWithMeans **item-based** algorithm, with the following parameters:
    - options for k: `[10, 20, 30, 40, 50]`
    - `'sim_options': {'name': ['pearson'], 'user_based': [???]}` Here you have to replace `???` with the appropriate value...
    - root-mean-square-error (RMSE) as measures,
    - 5 cross-validation folds,
    - other parameters: `refit=True, joblib_verbose=2, n_jobs=-1`
- What is the optimal k for which GridSearchCV returned the best RMSE score? 
- What is the RMSE score for the optimal k?

In [ ]:
# determine the parameters for item based filtering 
sim_k_grid = {"k": [10,20,30,40,50],
              "sim_options": {'name': ['pearson'], 'user_based': [False]}} # item based collaborative filtering
                            

#build a recommender system with GridSearchCV and fit the model
knn_cv = GridSearchCV(KNNWithMeans, sim_k_grid, measures = ["rmse"],cv=5, refit=True, joblib_verbose=2, n_jobs=-1)
knn_cv.fit(data_recomm)

In [ ]:
#Print results: 
print("Optimal k neighbour:", knn_cv.best_params["rmse"]["k"])
print("RMSE score for optimal k: ", round(knn_cv.best_score["rmse"],3))

- Using the Surprise library, split your dataset between training and test set. As parameters, use `test_size=0.2, random_state=12`
- Fit a KNNWithMeans algorithm using the best k value retrieved above. As other parameters, use:
    - `min_k=1`
    - `sim_options = {'name': 'pearson','user_based': ???}`
    - `verbose=False`
- Predict ratings on the test set using your algorithm

In [ ]:
#create a training and test set
recomm_train,recomm_test = train_test_split(data_recomm, test_size=0.2, random_state=12)

#create KnnMeans algorithm and fit train data
knn_means = KNNBasic(k= 50, min_k=1,sim_options= {'name': 'pearson','user_based': False},verbose=False)
knn_means.fit(recomm_train)

#predict ratings
predictions = knn_means.test(recomm_test)
#print(predictions)

- Use the helper function below to identify the best 10 films for all users
- Find the top 10 predictions for user 169 (you should return the titles of the movies)

In [ ]:
def read_item_names():
    '''Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    '''

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list) # This is used to group a sequence of key-value pairs into a dictionary of lists
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [ ]:
# identifying the best 10 films for all users
#display(read_item_names())

# get user ID's in a list
top_10 = get_top_n(predictions, n=10)
user_id = list(top_10.keys())
#display(user_id)

# get user movie names in a list
movie_names = list(top_10.values())
#display(movie_names)

# applying the helper function
rid_to_name, name_to_rid = read_item_names()

# create list for top 10 movies of each user
top_10_per_user = []

for i in user_id: 
    top_10_user = [i]
    for j in range(len(top_10[i])):
        movie_names = [rid_to_name[top_10[i][j][0]]],
        rating_user = round(top_10[i][j][1],2)
      #  top_ten_per_user.append(movie_names)
        top_10_user.append(rid_to_name[top_10[i][j][0]]), round(top_10[i][j][1],3)
    top_10_per_user.append(top_10_user)

#show top 10 in Panda Dataframe
df_top_10_per_user = pd.DataFrame(top_10_per_user)
df_top_10_per_user =  df_top_10_per_user.rename(columns={0: "user_ID"})
display(df_top_10_per_user.sort_values("user_ID").reset_index(drop=True))

In [ ]:
# top 10 film predictions for user 169
user_id = "169"
user_rating = top_10[user_id]
#sort descending - the movie with the highest rating on the top
sorted_user_rating = sorted(user_rating, key=lambda x: x[1],reverse=True)
for i in sorted_user_rating:
    print (rid_to_name[i[0]])


- Plot the precision at rank k and the recall at rank k on the same figure, for k between 0 and 20, and a relevance threshold of 3.75
- Plot the precision-recall curve

*You can, but do not have to, rely on the function(s) used in the lab (i.e., copying the code of the function(s))*

In [ ]:
#get function for precision and recall of my algorithm
def precision_recall_algo(algo):
    '''Return precision and recall at k metrics for an algorithm.'''    
    
    # Fit algo on training set
    algo.fit(recomm_train)
    
    # Predict on test set
    predictions = algo.test(recomm_test)
    
    # Compute precision and recall
    precision = []
    recall = []
    for k in range(21):
        precisions, recalls = precision_recall_at_k(predictions, k=k, threshold=3.75)
        precision.append( sum(prec for prec in precisions.values()) / len(precisions) )
        recall.append( sum(rec for rec in recalls.values()) / len(recalls) ) 
    
    return precision, recall

#get function for precision and recall for each user
def precision_recall_at_k(predictions, k=50, threshold=3.75):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [ ]:
#apply precision and recall function 
precision_val, recall_val = precision_recall_algo(knn_means)

# Plot the graph for recall and precision
plt.plot(range(21), recall_val, "green", label="Recall", marker = "8")
plt.plot(range(21), precision_val, 'red', label="Precision",marker = "8")
plt.legend()
plt.title("Precision and recall for item-based KNN")
plt.xlabel("K")
plt.xticks(np.arange(0, 21, step=1))
plt.ylabel("Score")
plt.show();

In [ ]:
# KNN precision and recall graph
#apply precision and recall function 
precision_val, recall_val = precision_recall_algo(knn_means)

# Plot
plt.step(recall_val, precision_val, color='g', where='post', label ='KNN (with k=50 and item-based)')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.title('Precision-Recall curve');

Congrats, you are done with the assignment! YAY